## Description
The objective of this competition is to create a machine learning model to detect fraudulent transactions.

Fraud detection is an important application of machine learning in the financial services sector. This solution will help Xente provide improved and safer service to its customers.

This competition is sponsored by Xente, Innovation Village, and insight2impact.

## Data
Xente is an e-commerce and financial service app serving 10,000+ customers in Uganda.

This dataset includes a sample of approximately 140,000 transactions that occurred between 15 November 2018 and 15 March 2019.

One of the challenges of fraud detection problems is that the data is highly imbalanced. 

Xente_variable_definitions.csv: Definition of the features per transaction
Training.csv: Transactions from 15 November 2018 to 13 February 2019, including whether or not each transaction is fraudulent. You will use this file to train your model.
Test.csv: Transactions from 13 February 2019 to 14 March 2019, not including whether or not each transaction is fraudulent. You will test your model on this file.
sample_submission.csv: is an example of what your submission file should look like. The order of the rows does not matter, but the names of the TransactionId must be correct. The value in FraudResult will be 1 for is a Fraud and 0 for is not a fraud.

## Evaluation
The error metric for this competition is the F1 score, which ranges from 0 (total failure) to 1 (perfect score). Hence, the closer your score is to 1, the better your model.

F1 Score: A performance score that combines both precision and recall. It is a harmonic mean of these two variables. Formula is given as: 2*Precision*Recall/(Precision + Recall)

Precision: This is an indicator of the number of items correctly identified as positive out of total items identified as positive. Formula is given as: TP/(TP+FP)

Recall / Sensitivity / True Positive Rate (TPR): This is an indicator of the number of items correctly identified as positive out of total actual positives. Formula is given as: TP/(TP+FN)

Where:

TP=True Positive
FP=False Positive
TN=True Negative
FN=False Negative

Info from Leaderboard: score to beat: 0,89

In [23]:
import pandas as pd
from datetime import datetime, date, time, timedelta

In [21]:
# Load the variable names
variable_meanings = pd.read_csv("data/variable meanings.csv")
pd.set_option('max_colwidth', 800)
variable_meanings

,Column Name,Definition
0,TransactionId,Unique ï¿½transaction identifier on platform
1,BatchId,Unique number assigned to a batch of transactions for processing
2,AccountId,Unique number identifying the customer on platform
3,SubscriptionId,Unique number identifying the customer subscription
4,CustomerId,Unique identifier attached to Account
5,CurrencyCode,Country currency
6,CountryCode,Numerical geographical code of country
7,ProviderId,Source provider of Item ï¿½bought.
8,ProductId,Item name being bought.
9,ProductCategory,ProductIds are organized into these broader product categories.


## Results from Pandas Profile
1. There is only one CurrencyCode, that means there is no additional information provided by this feature ==> Drop CurrencyCode
2. There is only one CountryCode, that means there is no additional information provided by this feature ==> Drop CountryCode
3. TransactionIDs are all distinct, that means there is no additional information provided ==> Drop TransactionIds
4. TransactionStartTime consist of timestamps. For further analysis ==> group them into timeframes (use "datetime")
5. Amount contains + and - values (due to debit/credit) ==> we need to create a column with debit credit and transform "amount" to absolut values
6. Extreeeemely imbalanced target value ==> Oversampling? ==> read further information / links auf zindi nutzen
7. Definition for column  CustomerID and AccountId seems to be mixed up

In [3]:
# Load the data
data_test = pd.read_csv("data/test.csv")
data_train = pd.read_csv("data/training.csv")

In [ ]:
data_train.head()
#data_train.shape

In [ ]:
frauds = data_train.query('FraudResult == 1')

In [ ]:
data_train.query('CustomerId == "CustomerId_4406" ')
and AccountId == "AccountId_3957"

In [ ]:
data_train.query('CustomerId == "CustomerId_3957" ')
#and AccountId == "AccountId_4406"

In [ ]:
data_train.query('Amount<0 and CustomerId== "CustomerId_4841"')

In [ ]:
data_train.info()

In [ ]:
data_train.Amount.describe()

In [ ]:
data_train.Amount.value_counts()

In [ ]:
data_train.Amount.plot.hist(bins=50, ylim=(0,200))

In [4]:
data_train.head()

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult
0,TransactionId_76871,BatchId_36123,AccountId_3957,SubscriptionId_887,CustomerId_4406,UGX,256,ProviderId_6,ProductId_10,airtime,ChannelId_3,1000.0,1000,2018-11-15T02:18:49Z,2,0
1,TransactionId_73770,BatchId_15642,AccountId_4841,SubscriptionId_3829,CustomerId_4406,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-20.0,20,2018-11-15T02:19:08Z,2,0
2,TransactionId_26203,BatchId_53941,AccountId_4229,SubscriptionId_222,CustomerId_4683,UGX,256,ProviderId_6,ProductId_1,airtime,ChannelId_3,500.0,500,2018-11-15T02:44:21Z,2,0
3,TransactionId_380,BatchId_102363,AccountId_648,SubscriptionId_2185,CustomerId_988,UGX,256,ProviderId_1,ProductId_21,utility_bill,ChannelId_3,20000.0,21800,2018-11-15T03:32:55Z,2,0
4,TransactionId_28195,BatchId_38780,AccountId_4841,SubscriptionId_3829,CustomerId_988,UGX,256,ProviderId_4,ProductId_6,financial_services,ChannelId_2,-644.0,644,2018-11-15T03:34:21Z,2,0


## Data cleaning

* Stripped the ID columns from non-integer characters and converted them to integers
* Separated TransactionStartTime into transactiontime and transactiondate
* Drop

In [22]:
def remove_letters(string):
    return int(string.split('_')[1])
    
id_columns = ["TransactionId","BatchId","AccountId","SubscriptionId","CustomerId","ProviderId","ProductId","ChannelId"]    
for i in id_columns:
    data_train[i] = data_train[i].apply(lambda x:remove_letters(x))

In [19]:
# seperate `TransactionStartTime` into time and date
def convert_to_date(date):
    # convert field into datetime format
    d = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
    # extract date
    return d.date()

def convert_to_time(date):
    d = datetime.strptime(date,'%Y-%m-%dT%H:%M:%SZ')
    # extract time
    return d.time()

In [20]:
# create new columns with seperate information for `TransactionTime` and `TransactionDate`
data_train['TransactionTime'] = data_train.TransactionStartTime.apply(lambda x: convert_to_time(x))
data_train['TransactionDate'] = data_train.TransactionStartTime.apply(lambda x: convert_to_date(x))

In [25]:
data_train

,TransactionId,BatchId,AccountId,SubscriptionId,CustomerId,CurrencyCode,CountryCode,ProviderId,ProductId,ProductCategory,ChannelId,Amount,Value,TransactionStartTime,PricingStrategy,FraudResult,TransactionTime,TransactionDate,DebitCredit
0,76871,36123,3957,887,4406,UGX,256,6,10,airtime,3,1000.0,1000,2018-11-15T02:18:49Z,2,0,02:18:49,2018-11-15,0
1,73770,15642,4841,3829,4406,UGX,256,4,6,financial_services,2,-20.0,20,2018-11-15T02:19:08Z,2,0,02:19:08,2018-11-15,1
2,26203,53941,4229,222,4683,UGX,256,6,1,airtime,3,500.0,500,2018-11-15T02:44:21Z,2,0,02:44:21,2018-11-15,0
3,380,102363,648,2185,988,UGX,256,1,21,utility_bill,3,20000.0,21800,2018-11-15T03:32:55Z,2,0,03:32:55,2018-11-15,0
4,28195,38780,4841,3829,988,UGX,256,4,6,financial_services,2,-644.0,644,2018-11-15T03:34:21Z,2,0,03:34:21,2018-11-15,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95657,89881,96668,4841,3829,3078,UGX,256,4,6,financial_services,2,-1000.0,1000,2019-02-13T09:54:09Z,2,0,09:54:09,2019-02-13,1
95658,91597,3503,3439,2643,3874,UGX,256,6,10,airtime,3,1000.0,1000,2019-02-13T09:54:25Z,2,0,09:54:25,2019-02-13,0
95659,82501,118602,4841,3829,3874,UGX,256,4,6,financial_services,2,-20.0,20,2019-02-13T09:54:35Z,2,0,09:54:35,2019-02-13,1
95660,136354,70924,1346,652,1709,UGX,256,6,19,tv,3,3000.0,3000,2019-02-13T10:01:10Z,2,0,10:01:10,2019-02-13,0


In [24]:
data_train['DebitCredit'] = data_train.Amount.apply(lambda x: 0 if x > 0 else 1)